<a href="https://colab.research.google.com/github/racoope70/exploratory_daytrading/blob/main/PPO_Quantconnect_Converted_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# main.py — QC consumer of external JSON signals (Gist/HTTP)
from AlgorithmImports import *
import json, math
from datetime import datetime, timezone, timedelta

class ExternalSignalConsumer(QCAlgorithm):

    # ======================= Init =======================
    def Initialize(self):
        self.SetStartDate(2024, 1, 1)
        self.SetEndDate(2024, 12, 31)
        self.SetCash(100000)

        # ---- Params (Project → Parameters) ----
        self.mode          = (self.GetParameter("Mode") or "json-live").strip().lower()      # json-live | json-ignore-time
        self.json_url      = (self.GetParameter("SignalsUrl") or "").strip()                 # REQUIRED
        self.symbols_param = (self.GetParameter("Symbols") or "UNH,TSLA,TMO").strip()
        self.poll_minutes  = int(self.GetParameter("PollingMinutes") or 60)

        self.ThrowIfEmptyUrl("SignalsUrl")

        # Add equities
        self.symbols = {}
        for tkr in [s.strip().upper() for s in self.symbols_param.split(",") if s.strip()]:
            self.symbols[tkr] = self.AddEquity(tkr, Resolution.Hour).Symbol

        # Slippage/brokerage optional
        self.SetSecurityInitializer(lambda sec: sec.SetSlippageModel(ConstantSlippageModel(0.01)))
        self.SetBrokerageModel(BrokerageName.INTERACTIVE_BROKERS_BROKERAGE, AccountType.MARGIN)

        # -------- Poll schedule (unchanged) --------
        self.Schedule.On(
            self.DateRules.EveryDay(),
            self.TimeRules.Every(timedelta(minutes=self.poll_minutes)),
            self.PollJsonAndTrade
        )

        # -------- NEW: metrics / charts / state --------
        # Execution chart will be created automatically when we first Plot into it.
        self.fill_count = 0                    # all fills (buys + sells)
        self.last_equity = float(self.Portfolio.TotalPortfolioValue)
        self.daily_returns = []                # end-of-day returns for Sharpe/PSR
        self.valid_until_epoch = None          # float seconds since epoch

        # push “order stats + risk” once a day, right after market close of SPY
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        self.Schedule.On(
            self.DateRules.EveryDay(self.spy),
            self.TimeRules.AfterMarketClose(self.spy, 1),
            self.PushDailyMetrics
        )

        self.Debug(f"Params OK | mode={self.mode} poll={self.poll_minutes} url={self.json_url[:60]}...")

    # ======================= Poll + Trade =======================
    def PollJsonAndTrade(self):
        # --- Download & parse ---
        raw = self.Download(self.json_url)
        if not raw:
            self.Debug("[WARN] Empty JSON from endpoint")
            return

        try:
            data = json.loads(raw)
        except Exception as e:
            self.Debug(f"[WARN] JSON parse error: {e}")
            return

        # --- freshness window (producer sets 'valid_until_utc') ---
        vu_aware = self.ParseUtcAny(data.get("valid_until_utc") or data.get("valid_until"))
        self.valid_until_epoch = self.ToUtcEpoch(vu_aware) if vu_aware else None

        # Compare using epoch seconds (no tz mismatch possible)
        if self.mode == "json-live" and self.valid_until_epoch:
            now_ep = self.ToUtcEpoch(self.UtcTime)
            if now_ep > self.valid_until_epoch:
                self.Debug(f"[STALE] now>{data.get('valid_until_utc')}, skip this poll")
                return

        # --- Pull latest model signals by symbol ---
        models = data.get("models") or []
        by_sym = {(m.get("symbol") or "").upper(): m for m in models}

        # --- Simple mapping to trades ---
        for tkr, sym in self.symbols.items():
            m = by_sym.get(tkr)
            if not m:
                continue

            signal = (m.get("signal") or "").upper()
            # confidence proxy: prefer explicit confidence, else |action|, else p_long
            conf = m.get("confidence")
            if conf is None:
                act = m.get("action")
                conf = abs(float(act)) if act is not None else float(m.get("p_long") or 0.0)
            conf = float(conf or 0.0)

            # position sizing rule of thumb
            target = min(0.5, max(0.1, conf))

            if signal == "BUY":
                self.SetHoldings(sym, target)
            elif signal == "SELL":
                self.Liquidate(sym)
            # HOLD → do nothing

            self.Debug(f"{tkr}: {signal} (conf={conf:.2f})")

    # ======================= Order events (NEW #1) =======================
    def OnOrderEvent(self, orderEvent: OrderEvent) -> None:
        if orderEvent.Status != OrderStatus.Filled:
            return

        self.fill_count += 1
        self.Plot("Execs", "Fills", self.fill_count)

        # use TradeBuilder to derive win/loss counts so far
        closed = list(self.TradeBuilder.ClosedTrades)
        wins   = sum(1 for t in closed if t.ProfitLoss >  0)
        losses = sum(1 for t in closed if t.ProfitLoss <= 0)
        self.Plot("Execs", "ClosedWins",  wins)
        self.Plot("Execs", "ClosedLosses", losses)

    # ======================= End-of-day return series (NEW #2) =======================
    def OnEndOfDay(self):
        # build daily return stream from total portfolio value
        cur = float(self.Portfolio.TotalPortfolioValue)
        if self.last_equity > 0:
            r = (cur / self.last_equity) - 1.0
            self.daily_returns.append(r)
            # keep last ~252 days to bound memory
            if len(self.daily_returns) > 400:
                self.daily_returns = self.daily_returns[-300:]
        self.last_equity = cur

    # Compute & push Sharpe/PSR once per day (scheduled in Initialize)
    def PushDailyMetrics(self):
        n = len(self.daily_returns)
        if n < 10:
            return

        mean = sum(self.daily_returns) / n
        # sample std-dev
        var = sum((x - mean) ** 2 for x in self.daily_returns) / max(n - 1, 1)
        sd = math.sqrt(max(var, 1e-12))

        # annualize assuming daily returns
        sharpe = (mean / sd) * math.sqrt(252.0) if sd > 0 else 0.0

        # very simple PSR versus S* = 0 (null Sharpe = 0)
        # Z = S * sqrt(n)  → PSR = Φ(Z)
        z = (mean / sd) * math.sqrt(n) if sd > 0 else 0.0
        psr = 0.5 * (1.0 + math.erf(z / math.sqrt(2.0)))

        self.Plot("Risk", "Sharpe(rolling)", sharpe)
        self.Plot("Risk", "PSR_vs0", psr)

        # also plot win-rate snapshot from TradeBuilder
        closed = list(self.TradeBuilder.ClosedTrades)
        if closed:
            wins = sum(1 for t in closed if t.ProfitLoss > 0)
            winrate = wins / float(len(closed))
            self.Plot("Risk", "WinRate", winrate)

    # ======================= Helpers =======================
    def ThrowIfEmptyUrl(self, which: str):
        if not self.json_url:
            raise ValueError(f"Parameter '{which}' not provided. Set it to your URL in Project → Parameters.")

    def ParseUtcAny(self, s):
        """Accept ISO8601 (with 'Z' or '+00:00') or epoch seconds; return tz-aware UTC datetime, or None."""
        if not s:
            return None
        s = str(s)
        # ISO path
        try:
            s2 = s.replace("Z", "+00:00")
            dt = datetime.fromisoformat(s2)
            if dt.tzinfo is None:
                return dt.replace(tzinfo=timezone.utc)
            return dt.astimezone(timezone.utc)
        except Exception:
            pass
        # epoch path
        try:
            return datetime.fromtimestamp(float(s), tz=timezone.utc)
        except Exception:
            return None

    def ToUtcEpoch(self, dt):
        """Return epoch seconds for either tz-naive (assumed UTC) or tz-aware datetime."""
        if dt is None:
            return None
        if isinstance(dt, datetime):
            if dt.tzinfo is None:
                dt = dt.replace(tzinfo=timezone.utc)
            return dt.timestamp()
        try:
            return float(dt)
        except Exception:
            return None
